In [4]:
#Imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
import tensorflow.keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout, BatchNormalization, MaxPooling2D, AveragePooling2D, Flatten, Conv2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from keras.models import load_model
import numpy as np
import pandas as pd
import matplotlib
import os
import math
#load mnist dataset
from keras.datasets import mnist

In [5]:
pip install split-folders

In [6]:
!unzip /content/trainPart1.zip

Archive:  /content/trainPart1.zip
   creating: train/
   creating: train/Sample002/
  inflating: train/Sample002/img002-048.png  
  inflating: train/Sample002/img002-050.png  
  inflating: train/Sample002/img002-006.png  
  inflating: train/Sample002/img002-004.png  
  inflating: train/Sample002/img002-053.png  
  inflating: train/Sample002/img002-045.png  
  inflating: train/Sample002/img002-052.png  
  inflating: train/Sample002/img002-047.png  
  inflating: train/Sample002/img002-020.png  
  inflating: train/Sample002/img002-009.png  
  inflating: train/Sample002/img002-011.png  
  inflating: train/Sample002/img002-013.png  
  inflating: train/Sample002/img002-034.png  
  inflating: train/Sample002/img002-038.png  
  inflating: train/Sample002/img002-024.png  
  inflating: train/Sample002/img002-025.png  
  inflating: train/Sample002/img002-018.png  
  inflating: train/Sample002/img002-032.png  
  inflating: train/Sample002/img002-028.png  
  inflating: train/Sample002/img002-001.pn

In [7]:
!splitfolders --output dataset_v1_again --ratio .8 .1 .1 -- /content/train

Copying files: 2480 files [00:00, 9338.40 files/s]


In [14]:
# Define the model1
# Passing a list of layers to the constructor
input_shape=(28,28,1) # size of input image and 
model = Sequential([
    Conv2D(64, (3,3), activation='relu',input_shape=input_shape),
    MaxPooling2D(pool_size=(2,2)), 
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),  
    Flatten(),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(62, activation='softmax'),])

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 32)        18464     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                25632     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)               

In [16]:
#train and val datagen
batch_size=32
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=0.2,
    horizontal_flip=True) # set validation split

train_generator = train_datagen.flow_from_directory(
    '/content/dataset_v1_again/train',
    target_size=(28, 28),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical') # set as training data
val_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = val_datagen.flow_from_directory(
    '/content/dataset_v1_again/val', # same directory as training data
    target_size=(28, 28),
    batch_size=batch_size,
     color_mode='grayscale',
    class_mode='categorical') # set as validation data

Found 1984 images belonging to 62 classes.
Found 248 images belonging to 62 classes.


In [17]:
#model compile with optimizer and parameters
model.compile(optimizer=tf.keras.optimizers.Adam(),# momentum=0.9, nesterov=True),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
#saving checkpoints
cp = tf.keras.callbacks.ModelCheckpoint(
    filepath='/content/drive/MyDrive/Task1_BestModel.h5',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [20]:
history=model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    callbacks=[cp],
    epochs =100,verbose=1)

Epoch 1/100
62/62 [==============================] - 60s 445ms/step - loss: 4.1467 - accuracy: 0.0230 - val_loss: 4.1329 - val_accuracy: 0.0179
Epoch 2/100
62/62 [==============================] - 27s 443ms/step - loss: 3.9610 - accuracy: 0.0571 - val_loss: 4.0981 - val_accuracy: 0.0223
Epoch 3/100
62/62 [==============================] - 28s 445ms/step - loss: 3.8325 - accuracy: 0.0583 - val_loss: 4.0540 - val_accuracy: 0.0312
Epoch 4/100
62/62 [==============================] - 28s 445ms/step - loss: 3.6861 - accuracy: 0.0978 - val_loss: 4.0118 - val_accuracy: 0.0357
Epoch 5/100
62/62 [==============================] - 28s 448ms/step - loss: 3.5583 - accuracy: 0.1258 - val_loss: 3.7862 - val_accuracy: 0.0759
Epoch 6/100
62/62 [==============================] - 28s 445ms/step - loss: 3.4511 - accuracy: 0.1457 - val_loss: 3.7000 - val_accuracy: 0.1205
Epoch 7/100
62/62 [==============================] - 27s 442ms/step - loss: 3.3137 - accuracy: 0.1671 - val_loss: 3.7993 - val_accuracy:

In [22]:
model.save('/content/drive/MyDrive/IIITDelhi_Task1_v1_again2_temp(52,54).h5')

In [23]:
#load model from previous saved best accuracy
model1=model
model1.load_weights('/content/drive/MyDrive/Task1_BestModel.h5')

In [24]:
#save model
model1.save('/content/drive/MyDrive/IIITDelhi_Task1_v1_again2_67.h5')